# Note de calcul de la puissance électrique nécessaire pour chauffer un tube calorifugé en fonction de la nature de son matériau

In [2]:
#Imports
import ipywidgets as widgets
from numpy import pi, log
import numpy as np
import matplotlib.pyplot as plt

## Dimensions du tubes
- La longueur du tube L_{section test} = $300$ mm
- Le diamètre intérieur $D_{int}$ =  $2,03$ mm
- Le diamètre extérieur $D_{ext}$ = $5,95$ mm
- La conductivité thermique $\lambda$ donnée par l'utilisat.eur.rice en $W.m^{-1}.K^{-1}$

### Entrées

In [3]:
L = 0.3 ; Dint = 2.03E-3 ; Dext = 5.95E-3
Sext = pi*Dext*L ; Sint = pi*Dint*L
Aint = pi*Dint**2/4 ; Aext = pi*Dext**2/4 
Vint = Aint*L

## Densité électrique et conductivité thermique

In [4]:
# Alliage (principal)
t = widgets.Label("Alliage",layout=widgets.Layout(display="flex", justify_content="center"))

e = widgets.Text(value='Résistivité électrique alliage [ohm.m]', disabled=True)
a = widgets.FloatText(min=1e-10, max=1e15, value = 250, description='rhoe_FT_a')
b = widgets.FloatSlider(min=1e-10, max=1e15, value = 250, description = 'rhoe_FS_a')

e1 = widgets.Text(value='Conductivité thermique alliage [W/K/m]', disabled=True)
a1 = widgets.FloatText(min=1e-10, max=1e15, value = 250)
b1 = widgets.FloatSlider(min=1e-10, max=1e15, value = 250)

r = widgets.Text(value=r'R_th = ' + str(0), disabled=True)
r1 = widgets.Text(value=r'R_elec = ' + str(0), disabled=True)

# Couche protectrice
t1 = widgets.Label("Couche protectrice",layout=widgets.Layout(display="flex", justify_content="center"))

e2 = widgets.Text(value='Epaisseur couche protectrice [um]', disabled=True)
a2 = widgets.FloatText(description='e_FT_cp',value=25)
b2 = widgets.FloatSlider(min=0, max=Dint*0.99999E6, description = 'e_FS_cp', value = 25)

e3 = widgets.Text(value='Résistivité électrique couche protectrice [ohm.m]', disabled=True)
a3 = widgets.FloatText(description='rhoe_FT_cp')
b3 = widgets.FloatSlider(min=1e-10, max=1e15,description = 'rhoe_FS_cp')

e4 = widgets.Text(value='Conductivité thermique couche protectrice [W/K/m]', disabled=True)
a4 = widgets.FloatText(min=1e-10, max=1e15, value = 25, decription='lmbda_FT_cp')
b4 = widgets.FloatSlider(min=1e-10, max=1e15, value = 25, description='lmbda_FS_cp')

r2 = widgets.Text(value=r'R_th = ' + str(0), disabled=True)
r3 = widgets.Text(value=r'R_elec = ' + str(0), disabled=True)

mylink  = widgets.jslink((a, 'value'), (b, 'value'))
mylink1  = widgets.jslink((a1, 'value'), (b1, 'value'))
mylink2  = widgets.jslink((a2, 'value'), (b2, 'value'))
mylink3  = widgets.jslink((a3, 'value'), (b3, 'value'))
mylink4 = widgets.jslink((a4, 'value'), (b4, 'value'))

vertical_box = widgets.VBox([t,e,a,b,e1,a1,b1,r,r1])
vertical_box1 = widgets.VBox([t1,e2,a2,b2,e3,a3,b3,e4,a4,b4,r2,r3])
sidetoside_box = widgets.HBox([vertical_box,vertical_box1])

#Propriétés de l'alliage
global rho_e, lmbda, R_a_elec, R_a_th
rho_e = a.value # Résistivité électrique de l'alliage
lmbda = a1.value # Conduction thermique de l'alliage
R_a_elec = rho_e*Sext/L
R_a_th = log(Dext/Dint)/(2*pi*lmbda*L)

# Propriétés de la couche protectrice
global e_cp, Sext_cp, rho_e_cp, lmbda_cp, R_cp_elec, R_cp_th, Aext_cp
e_cp = a2.value*1E-6 # [m]
Sext_cp = Sint # [m²]
Aext_cp = pi*((Dint/2)**2 - ((Dint/2)-e_cp)**2) # [m²]
rho_e_cp = a3.value
lmbda_cp = a4.value
R_cp_elec = rho_e_cp*Aext_cp/L
R_cp_th = log(Dint/(Dint-e_cp))/(2*pi*lmbda_cp*L)

#Interact

def handle_slider_change_Rth(change):
    global R_a_th
    h = help(change)
    R_a_th = log(Dext/Dint)/(2*pi*change.new*L)
    r1.value = 'Rth = ' + str(round(R_a_th,5)) + ' [K/Wth]'
b1.observe(handle_slider_change_Rth, names='value')

def handle_slider_change_Relec(change):
    global R_a_elec, change_name
    R_a_elec = change.new*Sext/L 
    r.value = 'Rélec = ' + str(round(R_a_elec,10)) + ' [ohms]'
b.observe(handle_slider_change_Relec, names='value')


def handle_slider_change_R_cp(change):
    global R_cp_th, Sext_cp, lmbda_cp, e_cp, R_cp_elec,Aext_cp, rho_e_cp
    if change.owner.description == 'e_FS_cp' :
        e_cp = change.new*1E-6
        Aext_cp = pi*((Dint/2)**2 - ((Dint/2)-e_cp)**2)
        R_cp_th = log(Dint/(Dint-e_cp))/(2*pi*lmbda_cp*L)
        R_cp_elec = rho_e_cp*Aext_cp/L
    elif change.owner.description == 'lmbda_FS_cp' :
        lmbda_cp = change.new
        R_cp_th = log(Dint/(Dint-e_cp))/(2*pi*lmbda_cp*L)
    elif change.owner.description == 'rhoe_FS_cp':
        rho_e_cp = change.new
        R_cp_elec = rho_e_cp*Aext_cp/L
    
    r2.value = 'Rth = ' + str(round(R_cp_th,5)) + ' [K/Wth]'
    r3.value = 'Relec = ' + str(round(R_cp_elec,20)) + ' [ohms]'
                   
b2.observe(handle_slider_change_R_cp, names='value')
b3.observe(handle_slider_change_R_cp, names='value')
b4.observe(handle_slider_change_R_cp, names='value')

display(sidetoside_box)


Help on Bunch in module traitlets.utils.bunch object:

class Bunch(builtins.dict)
 |  A dict with attribute-access
 |  
 |  Method resolution order:
 |      Bunch
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __dir__(self)
 |      Default dir() implementation.
 |  
 |  __getattr__(self, key)
 |  
 |  __setattr__(self, key, value)
 |      Implement setattr(self, name, value).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from builtins.dict:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, 